In [2]:
from pymongo import MongoClient

# Creation of a MongoDB Client (by giving the host and the port)
client = MongoClient(host="localhost", port=27017)

# Instantiation of the database
db = client["becode"]

# Let's see which collections are in the database
db.list_collection_names()
db.books.find_one()

{'_id': ObjectId('68b04cb1098efcd8f9ac1214'),
 'bookId': '2657.To_Kill_a_Mockingbird',
 'title': 'To Kill a Mockingbird',
 'series': 'To Kill a Mockingbird',
 'author': 'Harper Lee',
 'rating': 4.28,
 'description': 'The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it, To Kill A Mockingbird became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, To Kill A Mockingbird takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a mast

In [3]:
import ast

raw_genres = db["books"].find().distinct("genres")

print(raw_genres)

['10th Century', '11th Century', '12th Century', '13th Century', '14th Century', '15th Century', '16th Century', '17th Century', '18th Century', '19th Century', '1st Grade', '20th Century', '21st Century', '2nd Grade', '40k', 'Abuse', 'Academia', 'Academic', 'Academics', 'Action', 'Activism', 'Adolescence', 'Adoption', 'Adult', 'Adult Fiction', 'Adventure', 'Aeroplanes', 'Africa', 'African American', 'African American Literature', 'African American Romance', 'African Literature', 'Agriculture', 'Aircraft', 'Airships', 'Albanian Literature', 'Alchemy', 'Alcohol', 'Alexandria', 'Algebra', 'Algeria', 'Algorithms', 'Aliens', 'Alternate History', 'Alternate Universe', 'Alternative Medicine', 'Amateur Sleuth', 'Amazon', 'American', 'American Civil War', 'American Classics', 'American Fiction', 'American History', 'American Revolution', 'American Revolutionary War', 'Americana', 'Amish', 'Amish Fiction', 'Anarchism', 'Ancient', 'Ancient History', 'Angels', 'Anglo Saxon', 'Angola', 'Animal Fic

In [4]:
# find all books in dystopian genre
dystopian_books = db["books"].find({"genres": "Dystopia"})
print(len(list(dystopian_books)))

1692


In [16]:
kundera_books = db['books'].find({"author": {"$regex": "Milan Kundera"}}).sort("rating", -1)

for book in kundera_books:
    print(f"{book['title']}: {book['rating']}")

Immortality: 4.14
The Unbearable Lightness of Being: 4.1
Terra Nostra: 4.1
The Joke: 4.01
The Book of Laughter and Forgetting: 3.99
The Art of the Novel: 3.97
Life is Elsewhere: 3.95
Laughable Loves: 3.87
Farewell Waltz: 3.86
Ignorance: 3.78
Jacques and His Master: An Homage to Diderot in Three Acts: 3.77
Edward And God: 3.7
Identity: 3.68
Slowness: 3.65
The Festival of Insignificance: 3.33


In [27]:
def get_avg_rating_per_genre(genre):
    average_rating = list(db['books'].aggregate([
        {"$match": {"genres": genre}},
        {"$group": {"_id": None, "avgRating": {"$avg": "$rating"}}}
    ]))
    print(f"Average rating of {genre} books: {average_rating[0]['avgRating']:.2f}")

for genre in ['Dystopia', 'Art', 'Bicycles', 'Gothic', 'Magical Realism']:
    get_avg_rating_per_genre(genre)

Average rating of Dystopia books: 3.95
Average rating of Art books: 4.07
Average rating of Bicycles books: 4.21
Average rating of Gothic books: 3.86
Average rating of Magical Realism books: 3.90


In [ ]:
# genre with the highest average rating
result = list(db['books'].aggregate([
    {"$unwind": "$genres"},
    {"$group": {"_id": "$genres", "avgRating": {"$avg": "$rating"}}},
    {"$sort": {"avgRating": -1}},
    {"$limit": 10}
]))

if result:
    print("Book genres with highest average ratings:")
    for genre in result:
        print(f"{genre['_id']}: {genre['avgRating']:.2f}")
else:
    print("No genres found.")

Book genres with highest average ratings:
Colouring Books: 4.73
Aeroplanes: 4.71
Erotic Paranormal Romance: 4.69
Baha I: 4.62
Punx: 4.60
Omegaverse: 4.54
Cartoon: 4.47
Warriors: 4.46
Comic Strips: 4.46
Field Guides: 4.46
